<a href="https://colab.research.google.com/github/boiBASH/Tolaram_Project/blob/main/tolaram_recomender_unmasked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import pandas
import pandas as pd

# Step 3: Load your dataset (replace with the actual path in your Drive)
file_path = "/content/drive/MyDrive/TOLARAM/Data sample analysis.csv"
df = pd.read_csv(file_path, encoding='latin-1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-1-7cdf484f43a5>:10: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='latin-1')


In [2]:
# Step 1: Convert Delivered_date to datetime
df['Delivered_date'] = pd.to_datetime(df['Delivered_date'], errors='coerce', dayfirst=True)
df

,Brand,Branch,Designation,SKU_Code,Salesman_Code,Salesman_Name,Customer_Name,Customer_Phone,Order_Id,Delivered_date,Delivered Qty,Redistribution Value
0,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807259,ANOSIKE MIRACLE CHIMECHEFULAM,Eze store,8060733751,11432770,2024-08-05,3.0,"46,500"
1,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Chimaka,9016204667,11565436,2024-08-26,1.0,"15,500"
2,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Chimaka,9016204667,11727113,2024-09-17,1.0,"15,500"
3,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Cynthia chi,8060359775,11756045,2024-09-20,1.0,"15,500"
4,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Daddy Josephine,9075013565,11585122,2024-08-29,1.0,"15,500"
...,...,...,...,...,...,...,...,...,...,...,...,...
108820,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,House 1 super store,8101599308,12090661,2024-11-01,1.0,"56,800"
108821,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,Loveth ejiro,8139407516,12282897,2024-11-22,1.0,"33,000"
108822,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,Mrs Jennifer,8060468927,12067581,2024-10-30,4.0,"201,600"
108823,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,TWINNY SQUARE ENTERPRISES,8055148535,12032444,2024-10-25,1.0,"50,400"


In [3]:

# Step 2: Clean and convert Redistribution Value (remove commas and convert to float)
df['Redistribution Value'] = df['Redistribution Value'].str.replace(',', '', regex=False).astype(float)

# Step 3: Fill missing 'Delivered Qty' with 0 (or could be dropped if preferred)
df['Delivered Qty'] = df['Delivered Qty'].fillna(0)


# Confirm changes
df.dtypes


,0
Brand,object
Branch,object
Designation,object
SKU_Code,object
Salesman_Code,object
Salesman_Name,object
Customer_Name,object
Customer_Phone,int64
Order_Id,int64
Delivered_date,datetime64[ns]


In [4]:
# 1. Total amount spent per purchase
df['Total_Amount_Spent'] = df['Redistribution Value'] * df['Delivered Qty']

# 2. Average product cost per SKU
sku_grouped = df.groupby('SKU_Code').agg({
    'Redistribution Value': 'sum',
    'Delivered Qty': 'sum'
}).rename(columns={
    'Redistribution Value': 'Total_Value',
    'Delivered Qty': 'Total_Quantity'
})
sku_grouped['Average_Product_Cost'] = sku_grouped['Total_Value'] / sku_grouped['Total_Quantity']

# Merge back to main DataFrame
df = df.merge(sku_grouped[['Average_Product_Cost']], on='SKU_Code', how='left')
df

,Brand,Branch,Designation,SKU_Code,Salesman_Code,Salesman_Name,Customer_Name,Customer_Phone,Order_Id,Delivered_date,Delivered Qty,Redistribution Value,Total_Amount_Spent,Average_Product_Cost
0,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807259,ANOSIKE MIRACLE CHIMECHEFULAM,Eze store,8060733751,11432770,2024-08-05,3.0,46500.0,139500.0,15500.000000
1,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Chimaka,9016204667,11565436,2024-08-26,1.0,15500.0,15500.0,15500.000000
2,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Chimaka,9016204667,11727113,2024-09-17,1.0,15500.0,15500.0,15500.000000
3,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Cynthia chi,8060359775,11756045,2024-09-20,1.0,15500.0,15500.0,15500.000000
4,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Daddy Josephine,9075013565,11585122,2024-08-29,1.0,15500.0,15500.0,15500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108820,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,House 1 super store,8101599308,12090661,2024-11-01,1.0,56800.0,56800.0,49057.746991
108821,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,Loveth ejiro,8139407516,12282897,2024-11-22,1.0,33000.0,33000.0,49057.746991
108822,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,Mrs Jennifer,8060468927,12067581,2024-10-30,4.0,201600.0,806400.0,49057.746991
108823,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,TWINNY SQUARE ENTERPRISES,8055148535,12032444,2024-10-25,1.0,50400.0,50400.0,49057.746991


In [10]:
import pandas as pd
from pprint import pprint # Keep pprint for displaying the structured output

def analyze_customer_purchases_extended(df, customer_phone):
    customer_df = df[df['Customer_Phone'] == customer_phone].copy()

    if customer_df.empty:
        return f"No data found for customer phone: {customer_phone}"

    # Ensure date is sorted
    customer_df.sort_values('Delivered_date', inplace=True)

    # Add Month column
    customer_df['Month'] = customer_df['Delivered_date'].dt.to_period('M')

    # Basic customer information
    customer_name = customer_df['Customer_Name'].iloc[0] if not customer_df.empty else 'N/A'

    # 1. Total Brands Bought
    brands_bought = customer_df['Brand'].unique().tolist()
    total_brands_bought = len(brands_bought)

    # 2. SKUs of each Brand Bought
    brand_skus = customer_df.groupby('Brand')['SKU_Code'].unique().apply(list).to_dict()

    # 3. Purchase Summary by Brand
    purchase_summary_by_brand = {}
    for brand in brands_bought:
        brand_df = customer_df[customer_df['Brand'] == brand]
        last_purchase_date = brand_df['Delivered_date'].max().strftime('%Y-%m-%d') if not brand_df.empty else 'N/A'
        total_quantity = brand_df['Delivered Qty'].sum()
        total_spent = brand_df['Total_Amount_Spent'].sum()
        purchase_summary_by_brand[brand] = {
            'Last Purchase Date': last_purchase_date,
            'Total Quantity Bought': total_quantity,
            'Total Amount Spent': round(total_spent, 2)
        }

    # 4. Purchase Summary for Brand SKU
    purchase_summary_by_brand_sku = {}
    for brand, skus in brand_skus.items():
        purchase_summary_by_brand_sku[brand] = {}
        for sku in skus:
            sku_df = customer_df[(customer_df['Brand'] == brand) & (customer_df['SKU_Code'] == sku)]
            if not sku_df.empty:
                last_purchase = sku_df['Delivered_date'].max().strftime('%Y-%m-%d')
                # Handle potential division by zero if a Month group has no quantity/spend
                monthly_qty_series = sku_df.groupby('Month')['Delivered Qty'].sum()
                avg_monthly_qty = monthly_qty_series.mean().round(2) if not monthly_qty_series.empty else 0.0

                monthly_spend_series = sku_df.groupby('Month')['Total_Amount_Spent'].sum()
                avg_monthly_spend = monthly_spend_series.mean().round(2) if not monthly_spend_series.empty else 0.0

                purchase_summary_by_brand_sku[brand][sku] = {
                    'Last Purchase Date': last_purchase,
                    'Avg Monthly Quantity': avg_monthly_qty,
                    'Avg Monthly Spend': avg_monthly_spend
                }
            else:
                purchase_summary_by_brand_sku[brand][sku] = {
                    'Last Purchase Date': 'N/A',
                    'Avg Monthly Quantity': 0.0,
                    'Avg Monthly Spend': 0.0
                }


    # 5. Salesman Analysis
    # Handle case where customer_df might not have 'Salesman_Name' or 'Designation' columns
    most_sold_salesman = 'N/A'
    salesman_designation = 'N/A'

    if 'Salesman_Name' in customer_df.columns and not customer_df.empty:
        salesman_counts = customer_df.groupby(['Salesman_Name', 'Brand'])['Order_Id'].count().unstack(fill_value=0)
        if not salesman_counts.empty:
             most_sold_salesman = salesman_counts.sum(axis=1).idxmax() if salesman_counts.sum(axis=1).max() > 0 else 'N/A'

        # Assuming 'Designation' is the correct column name for salesman designation
        # Ensure 'Designation' column exists before accessing it
        if most_sold_salesman != 'N/A' and 'Designation' in customer_df.columns and not customer_df[customer_df['Salesman_Name'] == most_sold_salesman].empty:
             salesman_designation = customer_df[customer_df['Salesman_Name'] == most_sold_salesman]['Designation'].iloc[0]
        elif most_sold_salesman != 'N/A':
             salesman_designation = 'Designation data not available'


    # Format the output for better readability
    report = {
        'Customer Analysis Report': {
            'Customer Phone': customer_phone,
            'Customer Name': customer_name,
            'Overview': {
                'Total Unique Brands Bought': total_brands_bought,
                'Brands Bought List': brands_bought,
                'Top Salesperson (based on order count)': most_sold_salesman,
                'Salesperson Designation': salesman_designation
            },
            'Brand Level Summary': purchase_summary_by_brand,
            'Brand SKU Level Summary': purchase_summary_by_brand_sku,
            'SKUs Grouped by Brand': brand_skus # This is the same as SKUs by Brand
        }
    }


    return report

In [11]:
# Example of how to use the function and print the result
sample_customer = df['Customer_Phone'].iloc[70] # Use a valid customer phone from your df
analysis_report = analyze_customer_purchases_extended(df, sample_customer)
pprint(analysis_report)

{'Customer Analysis Report': {'Brand Level Summary': {'ADDMIE': {'Last Purchase Date': '2024-12-31',
                                                                 'Total Amount Spent': np.float64(23250.0),
                                                                 'Total Quantity Bought': np.float64(2.0)},
                                                      'INDOMIE': {'Last Purchase Date': '2024-12-07',
                                                                  'Total Amount Spent': np.float64(671700.0),
                                                                  'Total Quantity Bought': np.float64(17.0)},
                                                      'MINIMIE': {'Last Purchase Date': '2025-01-30',
                                                                  'Total Amount Spent': np.float64(840000.0),
                                                                  'Total Quantity Bought': np.float64(10.0)},
                                       

In [15]:
import pandas as pd
from IPython.display import display # Import display for better table rendering in notebooks

def analyze_customer_purchases_extended_table(df, customer_phone):
    customer_df = df[df['Customer_Phone'] == customer_phone].copy()

    if customer_df.empty:
        print(f"No data found for customer phone: {customer_phone}")
        return

    # Ensure date is sorted
    customer_df.sort_values('Delivered_date', inplace=True)

    # Add Month column
    customer_df['Month'] = customer_df['Delivered_date'].dt.to_period('M')

    # Basic customer information
    customer_name = customer_df['Customer_Name'].iloc[0] if not customer_df.empty else 'N/A'

    # 1. Total Brands Bought
    brands_bought = customer_df['Brand'].unique().tolist()
    total_brands_bought = len(brands_bought)

    # 5. Salesman Analysis
    most_sold_salesman = 'N/A'
    salesman_designation = 'N/A'

    if 'Salesman_Name' in customer_df.columns and not customer_df.empty:
        salesman_counts = customer_df.groupby(['Salesman_Name', 'Brand'])['Order_Id'].count().unstack(fill_value=0)
        if not salesman_counts.empty:
             most_sold_salesman = salesman_counts.sum(axis=1).idxmax() if salesman_counts.sum(axis=1).max() > 0 else 'N/A'

        # Assuming 'Designation' is the correct column name for salesman designation
        if most_sold_salesman != 'N/A' and 'Designation' in customer_df.columns and not customer_df[customer_df['Salesman_Name'] == most_sold_salesman].empty:
             salesman_designation = customer_df[customer_df['Salesman_Name'] == most_sold_salesman]['Designation'].iloc[0]
        elif most_sold_salesman != 'N/A':
             salesman_designation = 'Designation data not available'


    # --- Create Overview Table ---
    overview_data = {
        'Metric': ['Customer Phone', 'Customer Name', 'Total Unique Brands Bought', 'Top Salesperson', 'Salesperson Designation'],
        'Value': [
            customer_phone,
            customer_name,
            total_brands_bought,
            most_sold_salesman,
            salesman_designation
        ]
    }
    overview_df = pd.DataFrame(overview_data)

    print("--- Customer Overview ---")
    display(overview_df) # Use display for better formatting in notebooks

    print("\n--- Brands Bought ---")
    # Create a DataFrame for the list of brands bought
    brands_df = pd.DataFrame({'Brands Bought': brands_bought})
    display(brands_df)


    # --- Create Brand Level Summary Table ---
    print("\n--- Brand Level Summary ---")
    purchase_summary_by_brand_list = []
    for brand, summary in analyze_customer_purchases_extended(df, customer_phone)['Customer Analysis Report']['Brand Level Summary'].items():
         purchase_summary_by_brand_list.append({
             'Brand': brand,
             'Last Purchase Date': summary['Last Purchase Date'],
             'Total Quantity Bought': summary['Total Quantity Bought'],
             'Total Amount Spent': summary['Total Amount Spent']
         })
    brand_summary_df = pd.DataFrame(purchase_summary_by_brand_list)
    # Ensure Brand is the first column
    if 'Brand' in brand_summary_df.columns:
        cols = ['Brand'] + [col for col in brand_summary_df.columns if col != 'Brand']
        brand_summary_df = brand_summary_df[cols]
    display(brand_summary_df)


    # --- Create Brand SKU Level Summary Table(s) ---
    print("\n--- Brand SKU Level Summary ---")
    brand_sku_summary_data = analyze_customer_purchases_extended(df, customer_phone)['Customer Analysis Report']['Brand SKU Level Summary']
    for brand, skus_summary in brand_sku_summary_data.items():
        sku_summary_list = []
        for sku, summary in skus_summary.items():
            sku_summary_list.append({
                'SKU Code': sku,
                'Last Purchase Date': summary['Last Purchase Date'],
                'Avg Monthly Quantity': summary['Avg Monthly Quantity'],
                'Avg Monthly Spend': summary['Avg Monthly Spend']
            })
        sku_summary_df = pd.DataFrame(sku_summary_list)
        if not sku_summary_df.empty:
             print(f"\nBrand: {brand}")
             # Ensure 'SKU Code' is the first column
             if 'SKU Code' in sku_summary_df.columns:
                 cols = ['SKU Code'] + [col for col in sku_summary_df.columns if col != 'SKU Code']
                 sku_summary_df = sku_summary_df[cols]
             display(sku_summary_df)


    # --- Create SKUs Grouped by Brand Table(s) ---
    print("\n--- SKUs Grouped by Brand ---")
    skus_grouped_data = analyze_customer_purchases_extended(df, customer_phone)['Customer Analysis Report']['SKUs Grouped by Brand']
    for brand, skus in skus_grouped_data.items():
        if skus: # Check if the list of SKUs is not empty
             print(f"\nBrand: {brand}")
             skus_df = pd.DataFrame({'SKU Codes': skus})
             display(skus_df)

In [18]:
# Example of how to use the new table function
sample_customer = df['Customer_Phone'].iloc[899] # Use a valid customer phone from your df
analyze_customer_purchases_extended_table(df, sample_customer)

--- Customer Overview ---


,Metric,Value
0,Customer Phone,8164817950
1,Customer Name,Mr Oluwadamilola
2,Total Unique Brands Bought,7
3,Top Salesperson,LOLADE ABDULFATAH YUSUF
4,Salesperson Designation,FIELD EXECUTIVE



--- Brands Bought ---


,Brands Bought
0,COLGATE
1,HTC
2,INDOMIE
3,POWER OIL
4,KELLOGGS
5,MUNCH IT
6,ADDMIE



--- Brand Level Summary ---


...

,SKU Codes
0,10002832
1,10002773
2,10002830
3,10002838
4,10002843
5,10002771
6,10002823
7,10002826
8,10002835
9,10003378



Brand: MUNCH IT


,SKU Codes
0,10002849
1,10002861
2,10002860
3,10002866
4,10002857
5,10002837



Brand: POWER OIL


,SKU Codes
0,10000536
1,10000539
2,10000611
3,VOB4500_
4,10000533


In [9]:
sample_customer = df['Customer_Phone'].iloc[9999]
result = analyze_customer_purchases_extended(df, sample_customer)
from pprint import pprint
pprint(result)


{'Brands Bought': ['KELLOGGS', 'COLGATE', 'POWER OIL'],
 'Customer Phone': np.int64(8136149255),
 'Purchase Summary by Brand': {'COLGATE': {'Last Purchase Date': '2025-01-10',
                                           'Total Amount Spent': np.float64(128100.0),
                                           'Total Quantity Bought': np.float64(3.0)},
                               'KELLOGGS': {'Last Purchase Date': '2024-12-18',
                                            'Total Amount Spent': np.float64(301500.0),
                                            'Total Quantity Bought': np.float64(11.0)},
                               'POWER OIL': {'Last Purchase Date': '2024-12-20',
                                             'Total Amount Spent': np.float64(74000.0),
                                             'Total Quantity Bought': np.float64(2.0)}},
 'Purchase Summary by Brand SKU': {'COLGATE': {'10002804': {'Avg Monthly Quantity': np.float64(1.0),
                                    